# FALL 2020 -> CSE 6363 -> Machine Learning
## Assignment 2
### Problem 2
### Name:  Tirumala Manukonda (UTA ID# 1001662386)

#### Implementation multi-class logistic regression using a soft-max function and cross entropy
__Summary__:

- Read in Train and Test data set
- Using only images within class 0 to 5(for Train and Test Data)
- Implemented Multi-class Logistic regression using softmax and Cross entropy loss function
  - Softmax Regression is a form of logistic regression that normalizes an input value into 
    a vector of values that follows a probability distribution whose total sums up to 1.
  - The output values are between the range [0,1]
  - we are able to avoid binary classification and accommodate as many classes or dimensions 
    in our model
  -  softmax is sometimes referred to as a multinomial logistic regression.
- Classification report has given the values of our multi class logistic regression model
- Accuracy,Precision and Recall score are used for analysing the performance of our model


In [145]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import scipy.sparse

In [146]:
#Reading the csv file from the local folder and displaying top 10 values

Train_df = pd.read_csv("C:/Users/Dell/OneDrive/Documents/mnist_train.csv",sep=',',header=None) 
indexNames = Train_df[ Train_df[0] > 4].index
Train_df.drop(indexNames , inplace=True)
print(Train_df)


Test_df = pd.read_csv("C:/Users/Dell/OneDrive/Documents/mnist_test.csv",sep=',',header=None) 
indexNames = Test_df[ Test_df[0] > 4].index
Test_df.drop(indexNames , inplace=True)
print(Test_df)



       0    1    2    3    4    5    6    7    8    9    ...  775  776  777  \
1        0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
2        4    0    0    0    0    0    0    0    0    0  ...    0    0    0   
3        1    0    0    0    0    0    0    0    0    0  ...    0    0    0   
5        2    0    0    0    0    0    0    0    0    0  ...    0    0    0   
6        1    0    0    0    0    0    0    0    0    0  ...    0    0    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
59985    2    0    0    0    0    0    0    0    0    0  ...    0    0    0   
59987    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   
59991    2    0    0    0    0    0    0    0    0    0  ...    0    0    0   
59994    1    0    0    0    0    0    0    0    0    0  ...    0    0    0   
59996    3    0    0    0    0    0    0    0    0    0  ...    0    0    0   

       778  779  780  781  782  783  784  
1       

In [147]:
#assiging the class labels to y train and test data
X_train = Train_df.iloc[:,1:]
y_train =  Train_df[0]

X_test = Test_df.iloc[:,1:]
y_test =  Test_df[0]


In [148]:
#Softmax Function
#Compute the softmax of vector e

def softmax(e):
    e -= np.max(e)
    sm = (np.exp(e).T / np.sum(np.exp(e),axis=1)).T
    return sm

- The softmax function, also known as softargmax or normalized exponential function, is a generalization of the logistic  
  function to multiple dimensions
- In the softmax function we compute the exponential of the input parameter and the sum of exponential parameters of all 
  existing values in the inputs. Our output for the Softmax function is the ratio of the exponential of the parameter 
  and the   sum of  exponential parameter.
  

In [149]:
#converts integer class coding into one-hot varient

def oneHotEncoder(Y):
    m = Y.shape[0]
    #Y = Y[:,0]
    OHE = scipy.sparse.csr_matrix((np.ones(m), (Y, np.array(range(m)))))
    OHE = np.array(OHE.todense()).T
    return OHE

- oneHotEncoder function converts integer class coding, where there is a unidimensional array of labels into a one-hot 
  varient, where array is size of m * n

In [150]:
# To determine the probabilities and predictions
def getProbsAndPreds(someX):
    probs = softmax(np.dot(someX,w))
    preds = np.argmax(probs,axis=1)
    return probs,preds

- To determine the probabilities and predictions of each class for given input data

In [151]:
# Loss Function To compute loss and gradient
def LossFunction(w,x,y,lam):
    m = x.shape[0] 
    y_mat = oneHotEncoder(y) 
    scores = np.dot(x,w) 
    prob = softmax(scores) 
    loss = (-1 / m) * np.sum(y_mat * np.log(prob)) + (lam/2)*np.sum(w*w) 
    grad = (-1 / m) * np.dot(x.T,(y_mat - prob)) + lam*w 
    return loss,grad



- The loss function gets training examples, then one hot encoding of the data, computes the scores of input data
  and current weights,perform softmax to get probabilities of scores and then it computes the loss and gradient for the loss
- Loss or cost function denoted by y_mat means that only the output of the classifier corresponding to the correct class label
  is included in the cost. That is, when computing the cost for an example of the digit “4”, only the output of classifier
  4 contributes to the cost.
- If the classifier outputs 1 for the training example, then the cost is zero. The cost increases exponentially as 
  the classifier’s output decreases towards 0
    
    

In [152]:
#to get accuracy of the model
def getAccuracy(someX,someY):
    prob,prede = getProbsAndPreds(someX)
    accuracy = sum(prede == someY)/(float(len(someY)))
    return accuracy

- To calculate the accuracies of the model using the prediction of y and probability of the predictions are correct

In [168]:
#Calculating loss or cost, with values of lam =1, iterations = 100, learningRate = 0.001

w = np.zeros([X_train.shape[1],len(np.unique(y_train))])
lam = 1
iterations = 10000
learningRate = 0.0001
losses = []
for i in range(0,iterations):
    loss,grad = LossFunction(w,X_train,y_train,lam)
    losses.append(loss)
    w = w - (learningRate * grad)

print ('Training Accuracy: ', getAccuracy(X_train,y_train))
print ('Test Accuracy: ', getAccuracy(X_test,y_test))

Training Accuracy:  0.9708131781932279
Test Accuracy:  0.9729519361743529


- Using the max_iterations, Cost fuction calculates the loss and loss gradient on the training dataset
  and predicts the test set
- Training and testing accuracy are calculated which is 97% and 97%
  

In [169]:
#Classification report

y_pred_proba,y_pred = getProbsAndPreds(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.94      0.95      1032
           3       0.95      0.97      0.96      1010
           4       0.98      0.97      0.98       982

    accuracy                           0.97      5139
   macro avg       0.97      0.97      0.97      5139
weighted avg       0.97      0.97      0.97      5139



#### _Analysis_:- 
- My multiclass classification looking good with the good predictions of individual class labels 0,1,2,3,4 and overall good accuracy of 97 %, precision and recall also 97 %